<a href="https://colab.research.google.com/github/yoga-andri/phytoplankton-logistic-growth/blob/main/phytoplankton_logistic_growth_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Growth Model of Phytoplankton Based on Lighting Duration

This notebook simulates phytoplankton population growth using a logistic model,
based on varying light exposure durations (6–24 hours).

📊 **Features:**
- Interactive simulation using a slider
- Logistic curve prediction for chosen light exposure
- Comparison with actual data (6h, 12h, 24h)
- MAPE error analysis and interpretation

🔗 **Required file**: `logistic-growth-model-fitoplankton.xlsm` (must be uploaded to runtime)

Developed by: **[Yoga Andriyanto]**  
Date: **[July 2025]**

In [ ]:
# Install dependencies (for Colab)
!pip install ipywidgets openpyxl --quiet

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
from sklearn.metrics import mean_absolute_percentage_error

# Logistic model functions
def calculate_K(h, K_6, K_12, K_24):
    if h == 6:
        return K_6
    elif h == 12:
        return K_12
    elif h == 24:
        return K_24
    elif 6 < h < 12:
        return K_6 + (K_12 - K_6) / 6 * (h - 6)
    elif 12 < h < 24:
        return K_12 + (K_24 - K_12) / 12 * (h - 12)
    else:
        raise ValueError("Invalid light duration h. Must be 6–24.")

def calculate_r(h, a, b):
    return a * (1 - np.exp(-b * h))

def calculate_t0(h, p, q):
    return p * (1 - np.exp(-q * h))

def logistic_growth(t, K, r, t0):
    return K / (1 + np.exp(-r * (t - t0)))

# Load Excel data
data = pd.read_excel('logistic-growth-model-fitoplankton.xlsm', sheet_name="Data Fix")
days = data['Days']

# Parameters
K_6 = 2530000.000
K_12 = 4699444.444
K_24 = 7941388.889
a = 0.61999296
b = 0.12328981
p = 3.87674621
q = 0.18575130


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00


In [ ]:
def interactive_model(h_input):
    # Calculate logistic parameters
    K_new = calculate_K(h_input, K_6, K_12, K_24)
    r_new = calculate_r(h_input, a, b)
    t0_new = calculate_t0(h_input, p, q)

    # Generate predictions
    predictions = logistic_growth(days - 1, K_new, r_new, t0_new)

    # Calculate error (MAPE)
    mape_6 = mean_absolute_percentage_error(data['6 Hour'], predictions) * 100
    mape_12 = mean_absolute_percentage_error(data['12 Hour'], predictions) * 100
    mape_24 = mean_absolute_percentage_error(data['24 Hour'], predictions) * 100
    mape_values = [mape_6, mape_12, mape_24]
    mape_labels = ['6h', '12h', '24h']

    # --- Plot both logistic curve and error bar ---
    fig, axs = plt.subplots(2, 1, figsize=(8, 8))  # 2 rows, 1 column

    # Plot 1: Logistic Curve
    axs[0].scatter(days, data['6 Hour'], color='red', label='Actual Data 6 Hours')
    axs[0].scatter(days, data['12 Hour'], color='blue', label='Actual Data 12 Hours')
    axs[0].scatter(days, data['24 Hour'], color='green', label='Actual Data 24 Hours')
    axs[0].plot(days, predictions, color='black', linewidth=2, label=f'Logistic Prediction {h_input} Hours')
    axs[0].set_title(f'Logistic Growth Curve - {h_input} Hours Light Duration')
    axs[0].set_xlabel('Day')
    axs[0].set_ylabel('Density (cells/mL)')
    axs[0].legend()
    axs[0].grid(True)

    # Plot 2: Error Bar Chart
    axs[1].bar(mape_labels, mape_values, color=['red', 'blue', 'green'])
    axs[1].set_title('MAPE Compared to Actual Data')
    axs[1].set_ylabel('MAPE (%)')
    axs[1].grid(axis='y')

    plt.tight_layout()
    plt.show()

    # Print logistic parameters
    print(f"K value for {h_input} hours:  {K_new:.2f}")
    print(f"r value for {h_input} hours:  {r_new:.5f}")
    print(f"t0 value for {h_input} hours: {t0_new:.5f}")
    print("\nMAPE Compared to Actual Data:")
    print(f"- vs 6h  → {mape_6:.2f}%")
    print(f"- vs 12h → {mape_12:.2f}%")
    print(f"- vs 24h → {mape_24:.2f}%")

    # Interpretation
    errors = {'6h': mape_6, '12h': mape_12, '24h': mape_24}
    sorted_errors = sorted(errors.items(), key=lambda x: x[1])
    best_match, best_error = sorted_errors[0]
    second_best_match, second_best_error = sorted_errors[1]

    print("\n📌 Interpretation:")
    if abs(best_error - second_best_error) < 2:
        print(f"The prediction for {h_input} hours lies between {best_match} and {second_best_match} exposure.")
    else:
        print(f"The prediction for {h_input} hours is most similar to actual {best_match} exposure data.")


In [ ]:
# Slider for interactive input
interact(interactive_model, h_input=(6, 24, 1))


interactive(children=(IntSlider(value=15, description='h_input', max=24, min=6), Output()), _dom_classes=('wid…

<function __main__.interactive_model(h_input)>